In [ ]:
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import tensorflow as tf
import tensorlayer as tl
import matplotlib.pyplot as plt
import os

# import our model and env
from env import PortfolioEnv
from ddpg import actor_critic ,replay_buffer, actor_critic_ts_cs
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')
#pd.show_versions()

In [ ]:
# prepare history 

stock1 = pd.read_csv('C:/Users/lee/Desktop/Trading_Result/Data/嘉化能源.csv', header=0, index_col = 0, parse_dates = True)
stock2 = pd.read_csv('C:/Users/lee/Desktop/Trading_Result/Data/氯碱化工.csv', header=0, index_col = 0, parse_dates = True)
stock3 = pd.read_csv('C:/Users/lee/Desktop/Trading_Result/Data/中国平安.csv', header=0, index_col = 0, parse_dates = True)
stock4 = pd.read_csv('C:/Users/lee/Desktop/Trading_Result/Data/伊利股份.csv', header=0, index_col = 0, parse_dates = True)
stock5 = pd.read_csv('C:/Users/lee/Desktop/Trading_Result/Data/贵州茅台.csv', header=0, index_col = 0, parse_dates = True)
stock6 = pd.read_csv('C:/Users/lee/Desktop/Trading_Result/Data/陕鼓动力.csv', header=0, index_col = 0, parse_dates = True)
stock7 = pd.read_csv('C:/Users/lee/Desktop/Trading_Result/Data/双汇发展.csv', header=0, index_col = 0, parse_dates = True)


df = pd.concat([stock1, stock2, stock3, stock4, stock5, stock6, stock7], axis=1)
df.columns = pd.MultiIndex.from_product([["s1", "s2", "s3",'s4','s5','s6','s7'],['open', 'high', 'low', 'close', "volume"]], 
                                        names=['stock', 'price'])
df = df.dropna()

start = datetime.strptime("2013-10-22", '%Y-%m-%d' )
df_ = df.loc[start:]

# set the env input
history = df_ # the stock data
abbreviation = ["s1", "s2", "s3",'s4','s5','s6','s7'] # name of stock
steps = 200 # step for one eps
trading_cost = 0.0025
time_cost=0.00 # cost of holding equity
window_length = 10 # for obs
eps_move = 5 # move the start date after each rest
sample_start_date = "2016-05-01"

# build the replay_buffer
buffer_size = 100 # the max size of buffer

# build the actor, critic, and target network 
gamma = 0.99986 # discount factor
state_shape = [None,len(abbreviation),window_length,5] # (none, m_stock, history_window, feature)
action_shape = [None,len(abbreviation)] # (none, m_stock)
a_learning_rate = 0.01 # learning rate for actor
target_lr = 0.01 # learning rate for target, for stable
c_learning_rate = 0.01 # learning rate for critic

In [ ]:
import yfinance as yf

stocks =  ['amt', 'axp', 'ba', 'cvx', 'jnj', 'ko', 'mcd', 'msft', 't', 'wmt']
start = datetime(2012,1,1)
end = datetime(2022,11,11)
data = yf.download(stocks, start=start, end=end)
data_close = data['Adj Close']
data_close.plot()


# using US stock


df = data.drop('Close', axis=1)
df = df[['Open','High','Low','Adj Close','Volume']]
df.columns = df.columns.swaplevel(0, 1)
df.sort_index(axis=1, level=0, inplace=True)
df.columns = pd.MultiIndex.from_product([stocks,['open', 'high', 'low', 'close', "volume"]], 
                                        names=['stock', 'price'])
df = df.dropna()

start = datetime.strptime("2012-04-01", '%Y-%m-%d' )
df_ = df.loc[start:]

# set the env input
history = df_ # the stock data
abbreviation = stocks # name of stock
steps = 200 # step for one eps
trading_cost = 0.0025
time_cost=0.00 # cost of holding equity
window_length = 10 # for obs
eps_move = 0 # move the start date after each rest
sample_start_date = "2015-05-01"

# build the replay_buffer
buffer_size = int(steps * 5) # the max size of buffer

# build the actor, critic, and target network 
gamma = 1 # discount factor
state_shape = [None,len(abbreviation),window_length,5] # (none, m_stock, history_window, feature)
action_shape = [None,len(abbreviation)] # (none, m_stock)
a_learning_rate = 0.01 # learning rate for actor
target_lr = 0.001 # learning rate for target, for stable
c_learning_rate = 0.001 # learning rate for critic

alpha = 0
beta = 0
gamma_ = 0.5


In [ ]:
data_close['2016-02-01':'2017-04-01'].plot()

In [ ]:

RANDOM_SEED = 1234

# build the env
#PM_env = PM_env = PortfolioEnv.PortfolioEnv(history, abbreviation, steps, trading_cost, \
                                   #time_cost, window_length, eps_move, sample_start_date,\
                                   #alpha, beta, gamma_
                                  #)
# build the replay buffer
#rb = replay_buffer.ReplayBuffer(buffer_size, RANDOM_SEED)

# PM_env.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

state_dim = PM_env.observation_space.shape
action_dim = PM_env.action_space.shape
action_range = PM_env.action_space.high  # scale action, [-action_range, action_range]

# Training

In [ ]:
import matplotlib.pyplot as plt
import os


# build out agents
#  gamma, state_shape, action_shape, a_learning_rate, target_lr, c_learning_rate
ac = actor_critic.Actor_Critic_NetWork(gamma, state_shape,action_shape,a_learning_rate,target_lr,c_learning_rate)


TRAIN_EPISODES = 50

batch_size = int(buffer_size/4) # mini batch

train = True
RENDER = False

ALG_NAME = "DDPG"
ENV_ID = "Portfolio_Env"



t0 = time.time()
if train:  # train
    all_episode_reward = []
    for episode in range(TRAIN_EPISODES):
        state, info = PM_env.reset() 
        episode_reward = 0
        for step in range(steps):
            #print(state)
            if RENDER:
                PM_env.render()
            # Add exploration noise
            action = ac.Generate_action(state, greedy = False) #[[a0, a1, a2]]
            #print(action)
            state_, reward, done, info = PM_env.step(action) # compute the immidate reward and move to next steps
            #print(info)
            rb.add(state, action, reward, state_) # replay buffer add memory

            if rb.count > buffer_size:
                inputs = rb.sample_batch(batch_size)
                ac.learn(inputs)

            state = state_
            episode_reward += reward * gamma**step
            if done:
                break

        if episode == 0:
            all_episode_reward.append(episode_reward)
        else:
            all_episode_reward.append(all_episode_reward[-1] * 0.9 + episode_reward * 0.1)
        print(
            'Training  | Episode: {}/{}  | Episode Reward: {:.4f}  | Running Time: {:.4f}'.format(
                episode + 1, TRAIN_EPISODES, episode_reward,
                time.time() - t0
            )
        )
    ac.save()
    plt.plot(all_episode_reward)
    if not os.path.exists('image'):
        os.makedirs('image')
    plt.savefig(os.path.join('image', '_'.join([ALG_NAME, ENV_ID])))

# Test our model

test = True
TEST_EPISODES = 1
TEST_steps = steps


# right after the end of train data
#test_start_date = str(history.loc[sample_start_date:].index[steps])
#test_start_date= test_start_date[0:10] # only need year-month-day
test_start_date = sample_start_date


PM_env = PortfolioEnv.PortfolioEnv(history, abbreviation, steps, trading_cost, \
                                   time_cost, window_length, eps_move, sample_start_date,\
                                   alpha, beta, gamma_
                                  )

weight_track = {}
for name in abbreviation:
    weight_track[name] = []
    

if test:
    # test
    ac.load() # load previous parameters
    win_case = [] #used to count when cnn_model beat the equal weight portfolio
    for episode in range(TEST_EPISODES):
        state, info = PM_env.reset()
        episode_reward = 0
        for step in range(TEST_steps):
            action = ac.Generate_action(state, greedy = True)
            print(max(action[0]))
            state, reward, done, info = PM_env.step(action)
            #print(info)
            for i in range(len(abbreviation)):
                name = abbreviation[i]
                weight_track[name].append(action[0][i])
            episode_reward += reward * gamma**step
            if done:
                break
        if PM_env.infos[-1]["portfolio_value"] > PM_env.infos[-1]["equal_weight_portfolio_value"]:
            win_case.append(1) # if the CNN portfolio win
        else:
            win_case.append(0)
        PM_env.render()
        print(
            'Testing  | Episode: {}/{}  | Episode Reward: {:.4f}  | Running Time: {:.4f}'.format(
                episode + 1, TEST_EPISODES, episode_reward,
                time.time() - t0
            )
        )

In [ ]:
# the weight plot



x = np.arange(0,steps,1)
for name in abbreviation:
    y = weight_track[name]
    plt.plot(x, y, label = name, linestyle="-.")   
plt.legend()
plt.show()

# Forward Stepping Test

In [ ]:
eps_move = 5


ac = actor_critic.Actor_Critic_NetWork(gamma, state_shape,action_shape,a_learning_rate,target_lr,c_learning_rate)
warm_up = 15 # first 15 eps is for warm up only, test will perform after the warm up process
warm_up_count = 0

TRAIN_EPISODES = 65

batch_size = int(buffer_size/10) # mini batch

train = True
RENDER = False

ALG_NAME = "DDPG"
ENV_ID = "Portfolio_Env"

# trainning env
PM_env = PortfolioEnv.PortfolioEnv(history, abbreviation, steps, trading_cost, \
                                   time_cost, window_length, eps_move, sample_start_date,\
                                   alpha, beta, gamma_
                                  )
rb = replay_buffer.ReplayBuffer(buffer_size, RANDOM_SEED)

test = True
TEST_EPISODES = TRAIN_EPISODES - warm_up
TEST_step = eps_move # after the model is trained, n steps to test the data
TEST_steps = TEST_step * TEST_EPISODES # total steps for test period


# right after the end of train data
test_start_date = str(history.loc[sample_start_date:].index[int(steps + warm_up * eps_move)])
test_start_date= test_start_date[0:10] # only need year-month-day

# test env
test_PM_env = PortfolioEnv.PortfolioEnv(history, abbreviation, \
                                        TEST_steps, trading_cost, time_cost, \
                                        window_length, eps_move, test_start_date, \
                                        alpha, beta, gamma_
                                       )

weight_track = {}
bp_weight_track = {}
weighted_rank = []
for name in abbreviation:
    weight_track[name] = []
    bp_weight_track[name] = []
    

t0 = time.time()
if train:  # train
    test_state, test_info = test_PM_env.reset()
    for episode in range(TRAIN_EPISODES):
        ###################################### Train the model ############################################
        state, info = PM_env.reset()
        episode_reward = 0
        for step in range(steps):
            if RENDER:
                PM_env.render()
            # Add exploration noise
            action = ac.Generate_action(state, greedy = False) #[[a0, a1, a2]]
            state_, reward, done, info = PM_env.step(action)
            rb.add(state, action, reward, state_)

            if rb.count > buffer_size:
                inputs = rb.sample_batch(batch_size)
                ac.learn(inputs)

            state = state_
            if done:
                ac.save()
                warm_up_count += 1
                break
        ###################################### Test the model ############################################
        # test
        if (warm_up_count > warm_up):
            ac.load() # load previous parameters
            for step in range(TEST_step):
                test_action = ac.Generate_action(test_state, greedy = True)
                test_state, reward, done, info = test_PM_env.step(test_action)
                for i in range(len(abbreviation)):
                    name = abbreviation[i]
                    weight_track[name].append(test_action[0][i])
                    bp_weight_track[name].append(test_PM_env.sim.bp_weight[i])
                weighted_rank.append(test_PM_env.sim.weighted_rank)
                if done:
                    break
                episode_reward += reward * gamma**step
            print(
                'Testing  | Episode: {}/{} | Episode Reward: {:.4f}   | Running Time: {:.4f}'.format(
                    episode + 1, TRAIN_EPISODES, 
                    episode_reward,
                    time.time() - t0
                )
            )
        else:
            print(f'Warm up process: Episode {episode + 1}')
    test_PM_env.render()       
    ac.save()

In [ ]:
# the weight plot, show the dynamic of portfolio

import matplotlib.pyplot as plt
import numpy as np


x = test_PM_env.date_track[0:TEST_steps]
for name in abbreviation:
    y = weight_track[name]
    plt.plot(x, y, label = name, linestyle="-.")

plt.xticks(rotation=45, ha='right')
plt.xlabel("Date")
plt.ylabel("Weight")
plt.legend(loc = 'upper right')
plt.title("DDPG Dynamic weight")
plt.show()

plt.plot(x, weighted_rank,  linestyle="-")
plt.xticks(rotation=45, ha='right')
plt.xlabel("Date")
plt.ylabel("Weighted Rank")
y = len(abbreviation) * (len(abbreviation) - 1)/ (2 * len(abbreviation))
plt.axhline(y = y, color = 'r', linestyle = '-')
#plt.legend(loc = 'upper right')
plt.title("Weighted Rank ")
plt.show()


for name in abbreviation:
    z = bp_weight_track[name]
    plt.plot(x, z, label = name, linestyle="-.")
plt.xticks(rotation=45, ha='right')
plt.xlabel("Date")
plt.ylabel("Weight")

plt.legend(loc = 'upper right')
plt.title("Best past stock ")
plt.show()   




In [ ]:
test_PM_env.table()

# Forward Stepping Test with CS analysis

In [ ]:
# Forward Stepping Test

eps_move = 5


ac = actor_critic_ts_cs.Actor_Critic_NetWork(gamma, state_shape,action_shape,a_learning_rate,target_lr,c_learning_rate)
warm_up = 15 # first 15 eps is for warm up only


warm_up_count = 0

TRAIN_EPISODES = 65


batch_size = int(buffer_size/10) # mini batch

train = True
RENDER = False

ALG_NAME = "DDPG"
ENV_ID = "Portfolio_Env"

# trainning env
PM_env = PortfolioEnv.PortfolioEnv(history, abbreviation, steps, trading_cost, \
                                   time_cost, window_length, eps_move, sample_start_date,\
                                   alpha, beta, gamma_
                                  )
rb = replay_buffer.ReplayBuffer(buffer_size, RANDOM_SEED)

test = True
TEST_EPISODES = TRAIN_EPISODES - warm_up
TEST_step = eps_move # after the model is trained, n steps to test the data
TEST_steps = TEST_step * TEST_EPISODES # total steps for test period


# right after the end of train data
test_start_date = str(history.loc[sample_start_date:].index[int(steps + warm_up * eps_move)])
test_start_date= test_start_date[0:10] # only need year-month-day


# test env
test_PM_env = PortfolioEnv.PortfolioEnv(history, abbreviation, \
                                        TEST_steps, trading_cost, time_cost, \
                                        window_length, eps_move, test_start_date, \
                                        alpha, beta, gamma_
                                       )

weight_track = {}
bp_weight_track = {}
weighted_rank = []
for name in abbreviation:
    weight_track[name] = []
    bp_weight_track[name] = []
    

t0 = time.time()
if train:  # train
    test_state, test_info = test_PM_env.reset()
    for episode in range(TRAIN_EPISODES):
        ###################################### Train the model ############################################
        state, info = PM_env.reset()
        episode_reward = 0
        for step in range(steps):
            if RENDER:
                PM_env.render()
            # Add exploration noise
            action = ac.Generate_action(state, greedy = False) #[[a0, a1, a2]]
            state_, reward, done, info = PM_env.step(action)
            rb.add(state, action, reward, state_)

            if rb.count > buffer_size:
                inputs = rb.sample_batch(batch_size)
                ac.learn(inputs)

            state = state_
            if done:
                ac.save()
                warm_up_count += 1
                break
        ###################################### Test the model ############################################
        # test
        if (warm_up_count > warm_up):
            ac.load() # load previous parameters
            for step in range(TEST_step):
                test_action = ac.Generate_action(test_state, greedy = True)
                test_state, reward, done, info = test_PM_env.step(test_action)
                for i in range(len(abbreviation)):
                    name = abbreviation[i]
                    weight_track[name].append(test_action[0][i])
                    bp_weight_track[name].append(test_PM_env.sim.bp_weight[i])
                weighted_rank.append(test_PM_env.sim.weighted_rank)
                if done:
                    break
                episode_reward += reward * gamma**step
            print(
                'Testing  | Episode: {}/{} | Episode Reward: {:.4f}   | Running Time: {:.4f}'.format(
                    episode + 1, TRAIN_EPISODES, 
                    episode_reward,
                    time.time() - t0
                )
            )
        else:
            print(f'Warm up process: Episode {episode + 1}')
    test_PM_env.render()       
    ac.save()

In [ ]:
# the weight plot, show the dynamic of portfolio

import matplotlib.pyplot as plt
import numpy as np


x = test_PM_env.date_track[0:TEST_steps]
for name in abbreviation:
    y = weight_track[name]
    plt.plot(x, y, label = name, linestyle="-.")
    
plt.xlabel("Date")
plt.ylabel("Weight")
plt.legend(loc = 'upper right')
plt.title("DDPG_CS Dynamic weight")
plt.show()

plt.plot(x, weighted_rank,  linestyle="-")
plt.xlabel("Date")
plt.ylabel("Weighted Rank")
y = len(abbreviation) * (len(abbreviation) - 1)/ (2 * len(abbreviation))
plt.axhline(y = y, color = 'r', linestyle = '-')
#plt.legend(loc = 'upper right')
plt.title("Weighted Rank ")
plt.show()


for name in abbreviation:
    z = bp_weight_track[name]
    plt.plot(x, z, label = name, linestyle="-.")
    
plt.xlabel("Date")
plt.ylabel("Weight")

plt.legend(loc = 'upper right')
plt.title("Best past stock ")
plt.show()   


In [ ]:
test_PM_env.table()